# Load Libraries

In [ ]:
!git clone https://github.com/michalkrawczyk/Arxiv_GPT_Summarizer.git
%cd Arxiv_GPT_Summarizer

!pip install -r requirements.txt

from IPython.display import clear_output
clear_output()
print("Libraries Installed")

Libraries Installed


In [ ]:
from arxiv_utils import download_paper_from_arxiv, download_recent_papers_by_querry, PaperData
from gpt_core import get_description_json, get_summary, PROMPTS, reload_prompts
from datasets import PaperDataset

from tqdm import tqdm

import json
import os
from shutil import move
from time import sleep

# Define Files to summary (Choose option)

In [ ]:
#@markdown Specify settings for downloading papers by search querry
#@markdown ---
#@markdown Note: This will download most recent papers found with querry
SEARCH_QUERRY = "Artificial Inteligence" #@param {type:"string"}
assert SEARCH_QUERRY, "Empty search quarry"
#@markdown ---
#@markdown Number of papers to download
#@markdown ---

NUMBER_OF_PAPERS = 3 #@param {type:"integer"}
assert NUMBER_OF_PAPERS > 0
#@markdown <br>

DOWNLOADED_PAPERS = download_recent_papers_by_querry(SEARCH_QUERRY, NUMBER_OF_PAPERS)
print("\nDownloaded papers:")
for p in DOWNLOADED_PAPERS:
  print(f"\t{p}")
SEARCH_TYPE = "QUERRY"



Downloaded papers:
	./2305.08854v1.Laughing_Matters__Introducing_Laughing_Face_Generation_using_Diffusion_Models.pdf
	./2305.08852v1.Python_Tool_for_Visualizing_Variability_of_Pareto_Fronts_over_Multiple_Runs.pdf
	./2305.08848v1.Small_Models_are_Valuable_Plug_ins_for_Large_Language_Models.pdf


<h3><b>or</b></h3>

In [ ]:
#@markdown Provide papers to download by Arxiv ID
#@markdown ---
#@markdown Each paper id must be separated by space (' ') or comma (',')
PAPERS_BY_ID = "2301.05586, 2305.04889" #@param {type:"string"}

PAPERS_BY_ID = PAPERS_BY_ID.replace(' ', ',').split(',')
PAPERS_BY_ID = [p for p in PAPERS_BY_ID if p]
DOWNLOADED_PAPERS = download_paper_from_arxiv(PAPERS_BY_ID)
print("\nDownloaded papers:")
for p in DOWNLOADED_PAPERS:
  print(f"\t{p}")

SEARCH_TYPE = "ID"


Downloaded papers:
	./2301.05586v1.YOLOv6_v3_0_A_Full_Scale_Reloading.pdf
	./2305.04889v1.Improving_Real_Time_Bidding_in_Online_Advertising_Using_Markov_Decision_Processes_and_Machine_Learning_Techniques.pdf


# Inference Examples

## Single file example usage

In [ ]:
download_paper_from_arxiv(["2301.05586v1"])

In [ ]:
paper = PaperData("./2301.05586v1.YOLOv6_v3_0_A_Full_Scale_Reloading.pdf")
get_description_json(paper)

In [ ]:
paper = PaperData("./2301.05586v1.YOLOv6_v3_0_A_Full_Scale_Reloading.pdf")
summary = get_summary_3(paper)
print(summary[0])

100%|██████████| 5/5 [04:20<00:00, 52.07s/it]

New Features:
- YOLOv6 v3.0 has numerous novel enhancements on the network architecture and the training scheme.
- The neck of the detector is renewed with a Bi-directional Concatenation (BiC) module to provide more accurate localization signals.
- Anchor-aided training (AAT) strategy is proposed to enjoy the advantages of both anchor-based and anchor-free paradigms without touching inference efficiency.
- YOLOv6 is deepened to have another stage in the backbone and the neck, which reinforces it to hit a new state-of-the-art performance on the COCO dataset at a high-resolution input.
- A new self-distillation strategy is involved to boost the performance of small models of YOLOv6.

New Strategies:
- Anchor-aided training (AAT) strategy is proposed to enjoy the advantages of both anchor-based and anchor-free paradigms without touching inference efficiency.
- A new self-distillation strategy is involved to boost the performance of small models of YOLOv6.

Problems:
- The YOLO community h

## Make List with short Summaries

In [ ]:
dataset = PaperDataset()
if SEARCH_TYPE == "QUERRY":
  dataset.search_and_add_papers(SEARCH_QUERRY, NUMBER_OF_PAPERS)
  # dataset.search_and_add_papers("Artificial Inteligence", 3)
else:
  dataset.add_papers_by_id(PAPERS_BY_ID)

print()
print("Dataset Papers:", dataset.list_of_papers)
print("Dataset Features:", dataset.list_data_fields)
print("Dataset New Features:", dataset.list_new_features)
print("Dataset Paper Categories:", dataset.list_values_by_field("Model Category"))
print()

# Filter valuable papers by keywoard
summaries = dataset.search_by_field_value("New Features", "Novel")

Updating list of papers: 0it [00:00, ?it/s]


Dataset Papers: dict_keys(['2305.08854v1.Laughing_Matters__Introducing_Laughing_Face_Generation_using_Diffusion_Models.pdf', '2305.08852v1.Python_Tool_for_Visualizing_Variability_of_Pareto_Fronts_over_Multiple_Runs.pdf', '2305.08848v1.Small_Models_are_Valuable_Plug_ins_for_Large_Language_Models.pdf'])
Dataset Features: ['Model Name', 'New Features', 'Year', 'Model Category', 'filepath', 'SOTA']
Dataset New Features: {'Novel model capable of generating realistic laughter sequences', 'Python package for empirical attainment surface visualization', 'Allows black-box LLMs to work with locally fine-tuned smaller models resulting in improved performance on supervised tasks. Enhances the capabilities of smaller models such as multilinguality and interpretability.'}
Dataset Paper Categories: {'NLP', 'Image Generation', 'unknown'}



In [ ]:
# Save short summaries in json file
with open("short_summary.json", 'w') as f:
  json.dump(summaries, f, indent=2)

## Make Packages with longer summary

In [ ]:

for p in tqdm(DOWNLOADED_PAPERS[:1]):
  try:
    paper = PaperData(p)
    summary = get_summary(paper)

    paper_dir = os.path.splitext(os.path.basename(p))[0].split('.')[-1]
    if not os.path.isdir(paper_dir):
      os.makedirs(paper_dir)
    
    move(p, f"{paper_dir}/{os.path.basename(p)}")
    with open(f"{paper_dir}/summary.txt", 'w') as f:
      f.write(summary[-1])
      
  except Exception as err:
    print(f"""Failed to summarize: {p}
            - {err}""")


In [ ]:
for p in DOWNLOADED_PAPERS:
  paper_directory = os.path.splitext(os.path.basename(p))[0].split('.')[-1]
  if os.path.isdir(paper_directory):
    # Shell commands sometimes don't work with 'f" '
    zipname = paper_directory + ".zip"
    !zip -rm $zipname $paper_directory 

## (Optional) Save Paper and summary in Google Drive or download

### Download

In [ ]:
from google.colab import files

for p in DOWNLOADED_PAPERS:
  zipfile = os.path.splitext(os.path.basename(p))[0].split('.')[-1] + ".zip"

  if os.path.isfile(zipfile):
    files.download(zipfile) 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Move to Google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

GOOGLE_DRIVE_OUTPUT_DIRECTORY = "/content/drive/MyDrive/"
for p in DOWNLOADED_PAPERS:
  zipfile = os.path.splitext(os.path.basename(p))[0].split('.')[-1] + ".zip"

  if os.path.isfile(zipfile):
    move(zipfile, os.path.join(GOOGLE_DRIVE_OUTPUT_DIRECTORY, zipfile))
    print(f"Moved {zipfile} to {GOOGLE_DRIVE_OUTPUT_DIRECTORY}")

## (Optional) Download short summary

In [ ]:
from google.colab import files

files.download("short_summary.json") 